### Tải thủ công các thư viện ngoài

In [2]:
# Comment this line of code if you already have those downloaded
!pip install --no-index --no-deps /kaggle/input/aes-hcmus-whl/pyphen-0.15.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/aes-hcmus-whl/textstat-0.7.3-py3-none-any.whl
# !pip install --no-index --no-deps /kaggle/input/aes-hcmus-whl/pyspellchecker-0.8.1-py3-none-any.whl
print("\nDone installing")

Processing /kaggle/input/aes-hcmus-whl/pyphen-0.15.0-py3-none-any.whl
pyphen is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/aes-hcmus-whl/textstat-0.7.3-py3-none-any.whl
textstat is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

Done installing


### Import những thư viện cần thiết

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Basic libary
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import random
import torch
import re
import optuna
import textstat
from optuna.samplers import TPESampler
# cmap = plt.cm.get_cmap('coolwarm')
import warnings
warnings.filterwarnings('ignore')


# Use for pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.metrics import cohen_kappa_score, f1_score, make_scorer

# Use for training model
from scipy.stats import randint
from nltk.tokenize import word_tokenize
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier 
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv
/kaggle/input/aes-hcmus-whl/pyproject_toml-0.0.10-py3-none-any.whl
/kaggle/input/aes-hcmus-whl/textstat-0.7.3-py3-none-any.whl
/kaggle/input/aes-hcmus-whl/pyspellchecker-0.8.1-py3-none-any.whl
/kaggle/input/aes-hcmus-whl/pyphen-0.15.0-py3-none-any.whl


### Đọc file

In [4]:
df_train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')

### Preprocess

In [5]:
essay_id_dropped = df_train['essay_id']
df_train = df_train.drop('essay_id', axis = 1)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  17307 non-null  object
 1   score      17307 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 270.5+ KB


In [6]:
df_train.head(5)

,full_text,score
0,Many people have car where they live. The thin...,3
1,I am a scientist at NASA that is discussing th...,3
2,People always wish they had the same technolog...,4
3,"We all heard about Venus, the planet without a...",4
4,"Dear, State Senator\n\nThis is a letter to arg...",3


In [7]:
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)

def dataPreprocessing(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r'\xa0', '', x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

In [8]:
def textstat_features(text):
    features = {}
    features['flesch_reading_ease'] = textstat.flesch_reading_ease(text)
    features['flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(text)
    features['smog_index'] = textstat.smog_index(text)
    features['coleman_liau_index'] = textstat.coleman_liau_index(text)
    features['automated_readability_index'] = textstat.automated_readability_index(text)
    features['dale_chall_readability_score'] = textstat.dale_chall_readability_score(text)
    features['difficult_words'] = textstat.difficult_words(text)
    features['linsear_write_formula'] = textstat.linsear_write_formula(text)
    features['gunning_fog'] = textstat.gunning_fog(text)
    features['text_standard'] = textstat.text_standard(text, float_output=True)
    features['spache_readability'] = textstat.spache_readability(text)
    features['mcalpine_efg_time'] = textstat.reading_time(text)
    features['syllablaw'] = textstat.mcalpine_eflaw(text)
    features['readinle_count'] = textstat.syllable_count(text)
    features['lexicon_count'] = textstat.lexicon_count(text)
    features['monosyllabcount'] = textstat.monosyllabcount(text)
    return features

In [9]:
# Number of words
import string
def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    df['word_count'] = df['full_text'].apply(lambda x: len(x.split()))

    # Length
    df['essay_length'] = df['full_text'].str.len()

    # Sentences count
    # Adding a new column 'sentences_count' that counts the sentences in 'full_text'
    df['sentences_count'] = df['full_text'].str.count(r'\.')

    # Paragraph count
    # Adding a new column 'paragraph_count' that counts the paragraphs in 'full_text'
    df['paragraph_count'] = df['full_text'].str.count(r'\n') + 1
    
    df["text_tokens"] = df["full_text"].apply(lambda x: word_tokenize(x))
    df["word_count"] = df["text_tokens"].apply(lambda x: len(x))
    df["unique_word_count"] = df["text_tokens"].apply(lambda x: len(set(x)))
    df.drop(columns=["text_tokens"], inplace=True)
    
    df["processed_text"] = df["full_text"].apply(lambda x: dataPreprocessing(x))
    df["text_tokens"] = df["processed_text"].apply(lambda x: word_tokenize(x))
    df["text_length_p"] = df["processed_text"].apply(lambda x: len(x))
    df["word_count_p"] = df["text_tokens"].apply(lambda x: len(x))
    df["unique_word_count_p"] = df["text_tokens"].apply(lambda x: len(set(x)))
    
    # Applying textstat features
    df['textstat_features'] = df['processed_text'].apply(textstat_features)
    textstat_df = pd.DataFrame(df['textstat_features'].tolist())
    df = pd.concat([df, textstat_df], axis=1)

    df.drop(columns=["processed_text", "text_tokens", "textstat_features"], inplace=True)
    
    return df

def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Loại bỏ xuống dòng
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = re.sub(r'\xa0', '', text)
    text = text.lower()  # Chuyển thành chữ thường
    return text

In [10]:
df_train = preprocess_df(df_train)
df_train['clean_text'] = df_train['full_text'].apply(clean_text)
df_train = df_train.drop('full_text', axis = 1)

In [11]:
df_train.head(5)

,score,word_count,essay_length,sentences_count,paragraph_count,unique_word_count,text_length_p,word_count_p,unique_word_count_p,flesch_reading_ease,...,linsear_write_formula,gunning_fog,text_standard,spache_readability,mcalpine_efg_time,syllablaw,readinle_count,lexicon_count,monosyllabcount,clean_text
0,3,545,2677,13,1,248,2640,539,227,58.69,...,13.000000,17.08,12.0,7.20,31.58,53.4,624,489,396,many people have car where they live the thing...
1,3,371,1669,19,9,168,1663,371,152,87.55,...,6.714286,7.48,7.0,3.92,19.57,25.7,398,332,275,i am a scientist at nasa that is discussing th...
2,4,605,3077,24,7,243,3065,605,231,65.15,...,15.500000,11.49,12.0,5.12,36.96,32.6,767,550,417,people always wish they had the same technolog...
3,4,511,2701,23,9,241,2674,502,223,58.62,...,15.750000,11.85,13.0,5.32,32.74,28.9,678,441,284,we all heard about venus the planet without al...
4,3,418,2208,15,11,156,2184,417,148,54.76,...,19.666667,12.61,13.0,5.61,26.62,35.6,561,372,240,dear state senator this is a letter to argue ...


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   score                         17307 non-null  int64  
 1   word_count                    17307 non-null  int64  
 2   essay_length                  17307 non-null  int64  
 3   sentences_count               17307 non-null  int64  
 4   paragraph_count               17307 non-null  int64  
 5   unique_word_count             17307 non-null  int64  
 6   text_length_p                 17307 non-null  int64  
 7   word_count_p                  17307 non-null  int64  
 8   unique_word_count_p           17307 non-null  int64  
 9   flesch_reading_ease           17307 non-null  float64
 10  flesch_kincaid_grade          17307 non-null  float64
 11  smog_index                    17307 non-null  float64
 12  coleman_liau_index            17307 non-null  float64
 13  a

### Cài đặt Pipeline

**Câu hỏi**: Pipeline là gì?

> Pipeline là một cách để mã hóa và tự động hóa các công đoạn và quy trình làm việc cần thiết để tạo ra một mô hình học máy. Pipeline bao gồm nhiều bước tuần tự thực hiện mọi thứ từ trích xuất dữ liệu (data extraction) và tiền xử lý dữ liệu (preprocessing data) cho đến huấn luyện và triển khai mô hình.

> Đối với các hệ thống sử dụng mô hình ML, thì quy trình pipeline là phần trung tâm của sản phẩm. Nó đóng gói toàn bộ các phương pháp xử lý dữ liệu tốt nhất để tạo ra một mô hình học máy phù hợp nhất cho một bộ dữ liệu cụ thể. Ngoài ra pipeline còn cho phép mô hình thực thi trên quy mô lớn. Một thiết kế pipeline end-to-end sẽ cho phép hệ thống của bạn cập nhật một cách thường xuyên các mô hình học máy một cách nhanh chóng.

In [13]:
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(3,6),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)

train_tfid = vectorizer.fit_transform([i for i in df_train['clean_text']])
dense_matrix = train_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns

In [14]:
df_train = pd.concat([df_train, df], axis = 1)
df_train.head(5)

,score,word_count,essay_length,sentences_count,paragraph_count,unique_word_count,text_length_p,word_count_p,unique_word_count_p,flesch_reading_ease,...,tfid_18991,tfid_18992,tfid_18993,tfid_18994,tfid_18995,tfid_18996,tfid_18997,tfid_18998,tfid_18999,tfid_19000
0,3,545,2677,13,1,248,2640,539,227,58.69,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01778
1,3,371,1669,19,9,168,1663,371,152,87.55,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
2,4,605,3077,24,7,243,3065,605,231,65.15,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,4,511,2701,23,9,241,2674,502,223,58.62,...,0.01548,0.015657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
4,3,418,2208,15,11,156,2184,417,148,54.76,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Columns: 19027 entries, score to tfid_19000
dtypes: float64(19013), int64(13), object(1)
memory usage: 2.5+ GB


In [16]:
text_df = df_train['clean_text']
df_train = df_train.drop('clean_text', axis=1).astype('float64')

In [ ]:
print("Start running...")
# Define transformers for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Update categorical and numerical columns
numerical_columns = df_train.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df_train.select_dtypes('object').columns

# Remove target variable from numerical columns
numerical_columns = numerical_columns.drop('score')

print("- Numerical collumns: ", numerical_columns)
print("\n- Categorical collumns: ", categorical_columns)

# Combine transformers using ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
    ],remainder = 'passthrough')

# Create a pipeline with the preprocessor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)])

In [ ]:
# Apply the pipeline to your dataset
X = df_train.drop(['score'], axis=1).astype('float32')
X = X.apply(pd.to_numeric, errors='coerce')
y = np.log(df_train['score'])

X_preprocessed = pipeline.fit_transform(X)
print("This Pipeline is done")

In [ ]:
# After Pipeline
# Features columns have been normalized using StandardScaler()
print(X_preprocessed)

### Chia thành các tập train và tập test cho mô hình

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, 
                                                    test_size=0.2, random_state=42)

print("Train set shape: ", X_train.shape, " ; ", y_train.shape)
print("Test set shape: ",X_test.shape, " ; " ,y_test.shape)

### Cài đặt chấm điểm dựa theo cách chấm điểm của cuộc thi

> Submissions are scored based on the quadratic weighted kappa, which measures the agreement between two outcomes. This metric typically varies from 0 (random agreement) to 1 (complete agreement).

Các bài nộp được tính điểm dựa trên kappa có trọng số bậc hai, thước đo sự phù hợp giữa hai kết quả. Số liệu này thường thay đổi từ 0 (thỏa thuận ngẫu nhiên) đến 1 (thỏa thuận hoàn chỉnh).

In [ ]:
# # idea from https://www.kaggle.com/code/rsakata/optimize-qwk-by-lgb/notebook#QWK-objective
def quadratic_weighted_kappa(y_true, y_pred):
#     y_pred_exp = np.exp(y_pred)  # Chuyển đổi từ log-space về giá trị thực
#     y_pred_exp = y_pred_exp.clip(1, 6).round()  # Giới hạn giá trị giữa 1 và 6 và làm tròn

#     y_true_exp = np.round(np.exp(y_true))  # Chuyển đổi từ log-space về giá trị thực
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

# metric and objective based on public notebooks
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess

a = 2.998
b = 1.092

### Huấn luyện mô hình

In [ ]:
# print("Start running...")

# # Create a scorer for QWK
# qwk_scorer = make_scorer(quadratic_weighted_kappa, greater_is_better=True)
# eval_results = []

# # Cài đặt KFold
# kfold = KFold(n_splits=5, shuffle=True, random_state=0)

# xgb_callbacks = [
#     xgb.callback.EvaluationMonitor(period=25),
#     xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True)
# ]

# xgb_regressor = xgb.XGBRegressor(
#     objective=qwk_obj,  # Use custom QWK objective function
#     n_estimators= 207,
#     learning_rate= 0.10704240620854825,
#     min_split_loss = 1,
#     max_depth= 6,
#     subsample= 0.6389081081835488,
#     max_bin = 337,
#     random_state=42,
#     num_leaves = 10,
#     extra_trees=True,
#     class_weight='balanced',
#     tree_method="hist"
# )

# # Define evaluation sets for early stopping and monitoring
# eval_set = [(X_test, y_test)]

# # Train the model
# xgb_regressor.fit(X_train, y_train)
# #                 eval_set=eval_set,
# #                 eval_metric=quadratic_weighted_kappa, 
# #                 callbacks=xgb_callbacks,
# #                 verbose=True)

In [ ]:
# print("Start running...")

# # Create a scorer for QWK
# qwk_scorer = make_scorer(quadratic_weighted_kappa, greater_is_better=True)
# eval_results = []

# # Cài đặt KFold
# kfold = KFold(n_splits=5, shuffle=True, random_state=0)

# lgbm_regressor = lgb.LGBMRegressor(
#     objective = qwk_obj,
#     num_leaves = 10,
#     min_data_in_leaf = 100,
#     max_depth = 6,
#     n_estimators = 248,
#     random_state = 42,
#     max_bin = 288,
#     num_iterations = 120,
#     bagging_freq = 12,
#     extra_trees = True,
#     class_weight = 'balanced')


# # Define evaluation sets for early stopping and monitoring
# eval_set = [(X_test, y_test)]

# # Train the model
# lgbm_regressor.fit(X_train, y_train)
# #                 eval_set=eval_set,
# #                 eval_metric=quadratic_weighted_kappa, 
# #                 callbacks=xgb_callbacks,
# #                 verbose=True)

In [ ]:
# # Make predictions on the validation set
# y_pred = xgb_regressor.predict(X_test)

# y_pred = np.exp(y_pred)  # Chuyển đổi từ log-space về giá trị thực
# y_pred = y_pred.clip(1, 6).round()  # Giới hạn giá trị giữa 1 và 6 và làm tròn

# y_test = np.round(np.exp(y_test))  # Chuyển đổi từ log-space về giá trị thực

# # Calculate the Cohen's kappa score
# score = quadratic_weighted_kappa(y_test, y_pred)
# print(f"Train QWK: {score:.3f}")

In [ ]:
# # Make predictions on the validation set
# y_pred = lgbm_regressor.predict(X_test)

# y_pred = np.exp(y_pred)  # Chuyển đổi từ log-space về giá trị thực
# y_pred = y_pred.clip(1, 6).round()  # Giới hạn giá trị giữa 1 và 6 và làm tròn

# y_test = np.round(np.exp(y_test))  # Chuyển đổi từ log-space về giá trị thực

# # Calculate the Cohen's kappa score
# score = quadratic_weighted_kappa(y_test, y_pred)
# print(f"Train QWK: {score:.3f}")

In [ ]:
# fold_idx = 1
# for train_ids, val_ids in kfold.split(X, y):

#     xgb_regressor = xgb.XGBRegressor(
#     objective=qwk_obj,  # Use custom QWK objective function
#     n_estimators= 207,
#     learning_rate= 0.10704240620854825,
#     min_split_loss = 1,
#     max_depth= 6,
#     subsample= 0.6389081081835488,
#     max_bin = 337,
#     random_state=42,
#     num_leaves = 10,
#     extra_trees=True,
#     class_weight='balanced',
#     tree_method="hist"
#     )

#     print("Bắt đầu train Fold ", fold_idx)

#     # Train model
#     xgb_regressor.fit(X[train_ids], y[train_ids],
#               batch_size=batch_size,
#               epochs=no_epochs,
#               verbose=1)

#     # Test và in kết quả
#     y_pred = xgb_regressor.predict(X[train_ids])
#     # Calculate the Cohen's kappa score
#     score = quadratic_weighted_kappa(y[train_ids], y_pred)
#     print("Đã train xong Fold ", fold_idx)
#     print(f"Train QWK: {score:.3f}")

#     # Sang Fold tiếp theo
#     fold_idx = fold_idx + 1

### Dự đoán kết quả bằng mô hình vừa huấn luyện

### Optuna

> An open source hyperparameter optimization framework to automate hyperparameter search

 **Câu hỏi**: Khó khăn gặp phải khi phải điều chỉnh tham số thủ công?
 
 - Việc chọn siêu tham số khi thực hiện training mô hình dường như đều dựa vào cảm tính và kinh nghiệm của người lập trình. Và điều nay không phải lúc nào cũng có thể làm cho mô hình đạt hiệu suất, kết quả tốt nhất . Còn đối với những lập trình viên chưa có kinh nghiệm lựa chọn siêu tham số thì đây thực sự là một thử thách khi đòi hỏi phải có kiến thức và kinh nghiệm nhất định.

 **Câu hỏi**: Optuna là gì ?
 
 - Optuna là 1 framework hỗ trợ việc tự động điều chỉnh tham số mô hình để mô hình có thể đạt được hiệu năng tốt nhất. Do đó giúp giảm bớt thời gian và công sức khi chúng ta không cần phải điều chỉnh các siêu tham số một cách thủ công nữa. Từ đó giúp tăng năng suất công việc.

#### Định nghĩa objective function

**Trial object definition**: 

> Trial là một đối tượng thể hiện của một class được thực thi trong Optuna.Nó được sử dụng để định nghĩa các siêu tham số được tối ưu. Giá trị được lấy trong phạm vi được định nghĩa, thông tin của các tham số được tìm kiếm trong quá khứ vẫn được giữ lại và giá trị mới sẽ dựa trên những thông tin đó. 

Hay dễ hiểu hơn là Optuna sẽ duyệt qua các tham số được cài đặt thử nghiệm và chọn tập hợp siêu tham số có kết quả tốt nhất là tham số sử dụng cho mô hình cuối cùng.

In [ ]:
# def objective(trial):
#     param_grid = {
#         'n_estimators': trial.suggest_int('n_estimators', 500, 700),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
#         'min_split_loss': trial.suggest_int('min_split_loss', 0, 9),
#         'max_depth': trial.suggest_int('max_depth', 6, 12),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'max_bin': trial.suggest_int('max_bin', 256, 512)
#     }

#     xgb_regressor = xgb.XGBRegressor(
#         objective=qwk_obj,
#         random_state=42,
#         tree_method="hist",
#         **param_grid
#     )
    
#     xgb_regressor.fit(X_train, y_train)
    
#     y_pred_log = xgb_regressor.predict(X_test)
#     y_pred = np.exp(y_pred_log)
#     # y_pred = y_pred + a
#     y_pred = y_pred.clip(1, 6).round()
#     y_test_exp = np.exp(y_test)

#     # Calculate the Cohen's kappa score
#     score = cohen_kappa_score(y_test_exp.round(), y_pred, weights='quadratic')
#     return np.mean(score)

In [ ]:
def objective(trial):
    param_grid = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 60, 120),
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'max_bin': trial.suggest_int('max_bin', 200, 400),
        'bagging_freq': trial.suggest_int('bagging_freq', 10, 14)
    }

    lgbm_regressor = lgb.LGBMRegressor(
        objective=qwk_obj,
        random_state=42,
        extra_trees = True,
         class_weight = 'balanced',
        **param_grid
    )
    
    lgbm_regressor.fit(X_train, y_train)
    
    y_pred_log = lgbm_regressor.predict(X_test)
    y_pred = np.exp(y_pred_log)
    # y_pred = y_pred + a
    y_pred = y_pred.clip(1, 6).round()
    y_test_exp = np.exp(y_test)

    # Calculate the Cohen's kappa score
    score = cohen_kappa_score(y_test_exp.round(), y_pred, weights='quadratic')
    return np.mean(score)

**Định nghĩa study object**: Để tìm kiếm siêu tham số, bạn cần khởi tạo một đối tượng là study 

> Đối tượng này lưu kết quả tối ưu của bạn.

**Sau đó, sử dụng phương thức optimize**:

- Trong đó, tham số thứ 1 là hàm Objective 
- Tham số thứ 2 là số lượng thử nghiệm. 

> Quá trình tối ưu này được thực hiện trong đối tượng Study và sẽ thực hiện tìm giá trị cực tiểu của các tham số trong hàm Objective bằng phương pháp tối ưu hóa. 

In [ ]:
# Set up the Optuna study
study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=4)

> Khi kết thúc quá trình trên, giá trị tối ưu sẽ được lưu lại và bạn có thể xem nó bằng câu lệnh sau:

In [ ]:
# Get the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

#### Thực hiện áp dụng các siêu tham số mà framework Optuna đã huấn luyện và chọn lọc

In [ ]:
# # Train the final model with the best hyperparameters
# xgb_regressor = xgb.XGBRegressor(
#     objective=qwk_obj,
#     random_state=42,
#     **best_params
# )

# xgb_regressor.fit(
#     X_train, y_train,
#     early_stopping_rounds=10,
#     eval_set=[(X_test, y_test)],
#     verbose=True
# )

# y_pred_log = xgb_regressor.predict(X_test)
# y_pred = np.exp(y_pred_log)
# # y_pred = y_pred + a
# y_pred = y_pred.clip(1, 6).round()
# y_test_exp = np.exp(y_test)

# # Calculate the Cohen's kappa score
# score = cohen_kappa_score(y_test_exp.round(), y_pred, weights='quadratic')
# print(f"Train QWK: {score}")

In [ ]:
# Train the final model with the best hyperparameters
lgbm_regressor = lgb.LGBMRegressor(
    objective=qwk_obj,
    random_state=42,
    **best_params
)

lgbm_regressor.fit(
    X_train, y_train,
    early_stopping_rounds=10,
    eval_set=[(X_test, y_test)],
    verbose=True
)

y_pred_log = lgbm_regressor.predict(X_test)
y_pred = np.exp(y_pred_log)
# y_pred = y_pred + a
y_pred = y_pred.clip(1, 6).round()
y_test_exp = np.exp(y_test)

# Calculate the Cohen's kappa score
score = cohen_kappa_score(y_test_exp.round(), y_pred, weights='quadratic')
print(f"Train QWK: {score}")

### Thực hiện áp dụng Model đã huấn luyện vào file test.csv

In [ ]:
df_test = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')
df_test

In [ ]:
df_test = preprocess_df(df_test)
df_test['clean_text'] = df_test['full_text'].apply(clean_text)
df_test = df_test.drop('full_text', axis = 1)
df_test.head()

### Xuất kết quả ra file submission và nộp bài

In [ ]:
# Lưu lại cột essay_id để sử dụng sau này
essay_ids = df_test['essay_id']

test_tfid = vectorizer.transform([i for i in df_test['clean_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns

df_test = pd.concat([df_test, df], axis = 1)
X_test = df_test.drop(['clean_text', 'essay_id'], axis=1).astype('float64')

X_test

In [ ]:
print(X_test.dtypes)

In [ ]:
# # Áp dụng pipeline đã huấn luyện để tiền xử lý dữ liệu test
# X_test_preprocessed = pipeline.transform(X_test)

# # Dự đoán điểm số trên dữ liệu test đã tiền xử lý
# y_pred_log = xgb_regressor.predict(X_test_preprocessed)

# # Chuyển đổi ngược từ log-transform
# y_pred = np.exp(y_pred_log)

# # Tạo dataframe chứa essay_id và dự đoán điểm số
# results = pd.DataFrame({'essay_id': essay_ids, 'score': y_pred})

# # Làm tròn điểm số
# results['score'] = round(results['score'])
# results['score'] = results['score'].astype(int)

# # Lưu kết quả ra file csv
# submission = results.to_csv('submission.csv', index=False)

In [ ]:
# Áp dụng pipeline đã huấn luyện để tiền xử lý dữ liệu test
X_test_preprocessed = pipeline.transform(X_test)

# Dự đoán điểm số trên dữ liệu test đã tiền xử lý
y_pred_log = lgbm_regressor.predict(X_test_preprocessed)

# Chuyển đổi ngược từ log-transform
y_pred = np.exp(y_pred_log)

# Tạo dataframe chứa essay_id và dự đoán điểm số
results = pd.DataFrame({'essay_id': essay_ids, 'score': y_pred})

# Làm tròn điểm số
results['score'] = round(results['score'])
results['score'] = results['score'].astype(int)

# Lưu kết quả ra file csv
submission = results.to_csv('submission.csv', index=False)

In [ ]:
results